<a href="https://colab.research.google.com/github/krakowiakpawel9/machine-learning-bootcamp/blob/master/unsupervised/05_case_studies/01_customer_segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### scikit-learn
Strona biblioteki: [https://scikit-learn.org](https://scikit-learn.org)  

Dokumentacja/User Guide: [https://scikit-learn.org/stable/user_guide.html](https://scikit-learn.org/stable/user_guide.html)

Podstawowa biblioteka do uczenia maszynowego w języku Python.

Aby zainstalować bibliotekę scikit-learn, użyj polecenia poniżej:
```
!pip install scikit-learn
```
Aby zaktualizować do najnowszej wersji bibliotekę scikit-learn, użyj polecenia poniżej:
```
!pip install --upgrade scikit-learn
```
Kurs stworzony w oparciu o wersję `0.22.1`

### Spis treści:
1. [Import bibliotek](#0)
2. [Załadowanie danych](#1)
3. [Eksploracja danych](#2)
4. [Wyznacznie retencji klienta](#3)
5. [Retencja - KMeans](#4)
6. [Retencja - DBSCAN](#5)
7. [Sprzedaż](#6)
8. [Sprzedaż - KMeans](#7)
9. [Sprzedaż - DBSCAN](#8)
10. [Retencja, sprzedaż - KMeans](#9)




### <a name='0'></a> Import bibliotek

In [0]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

### <a name='1'></a> Załadowanie danych

In [2]:
url = 'https://storage.googleapis.com/esmartdata-courses-files/ml-course/OnlineRetail.csv'
raw_data = pd.read_csv(url, encoding='latin', parse_dates=['InvoiceDate'])
data = raw_data.copy()
data.head(3)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom


### <a name='2'></a> Eksploracja danych

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
InvoiceNo      541909 non-null object
StockCode      541909 non-null object
Description    540455 non-null object
Quantity       541909 non-null int64
InvoiceDate    541909 non-null datetime64[ns]
UnitPrice      541909 non-null float64
CustomerID     406829 non-null float64
Country        541909 non-null object
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


In [4]:
data.describe()

,Quantity,UnitPrice,CustomerID
count,541909.000000,541909.000000,406829.000000
mean,9.552250,4.611114,15287.690570
std,218.081158,96.759853,1713.600303
min,-80995.000000,-11062.060000,12346.000000
25%,1.000000,1.250000,13953.000000
50%,3.000000,2.080000,15152.000000
75%,10.000000,4.130000,16791.000000
max,80995.000000,38970.000000,18287.000000


In [5]:
data.describe(include=['object'])

,InvoiceNo,StockCode,Description,Country
count,541909,541909,540455,541909
unique,25900,4070,4223,38
top,573585,85123A,WHITE HANGING HEART T-LIGHT HOLDER,United Kingdom
freq,1114,2313,2369,495478


In [6]:
data.describe(include=['datetime'])

,InvoiceDate
count,541909
unique,23260
top,2011-10-31 14:41:00
freq,1114
first,2010-12-01 08:26:00
last,2011-12-09 12:50:00


In [7]:
data.isnull().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

In [8]:
# usunięcie braków
data = data.dropna()
data.isnull().sum()

InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64

In [9]:
data['Country'].value_counts()

United Kingdom          361878
Germany                   9495
France                    8491
EIRE                      7485
Spain                     2533
Netherlands               2371
Belgium                   2069
Switzerland               1877
Portugal                  1480
Australia                 1259
Norway                    1086
Italy                      803
Channel Islands            758
Finland                    695
Cyprus                     622
Sweden                     462
Austria                    401
Denmark                    389
Japan                      358
Poland                     341
USA                        291
Israel                     250
Unspecified                244
Singapore                  229
Iceland                    182
Canada                     151
Greece                     146
Malta                      127
United Arab Emirates        68
European Community          61
RSA                         58
Lebanon                     45
Lithuani

In [10]:
tmp = data['Country'].value_counts().reset_index()
tmp.columns = ['Country', 'Count']
tmp.query("Count > 200", inplace=True)
px.bar(tmp, x='Country', y='Count', template='plotly_dark', color_discrete_sequence=['#03fcb5'],
       title='Częstotliwość zakupów ze względu na kraj')

In [11]:
# obcięcie tylko do United Kingdom
data_uk = data.query("Country == 'United Kingdom'").copy()
data_uk.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [12]:
# utworzenie nowej zmiennej Sales
data_uk['Sales'] = data_uk['Quantity'] * data_uk['UnitPrice']
data_uk.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Sales
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34


In [13]:
# częstotliwość zakupów ze względu na datę
tmp = data_uk.groupby(data_uk['InvoiceDate'].dt.date)['CustomerID'].count().reset_index()
tmp.columns = ['InvoiceDate', 'Count']
tmp.head()

,InvoiceDate,Count
0,2010-12-01,1809
1,2010-12-02,2029
2,2010-12-03,937
3,2010-12-05,2492
4,2010-12-06,1915


In [14]:
from plotly.subplots import make_subplots

fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.02)

trace1 = px.line(tmp, x='InvoiceDate', y='Count', template='plotly_dark', color_discrete_sequence=['#03fcb5'])['data'][0]
trace2 = px.scatter(tmp, x='InvoiceDate', y='Count', template='plotly_dark', color_discrete_sequence=['#03fcb5'])['data'][0]

fig.add_trace(trace1, row=1, col=1)
fig.add_trace(trace2, row=2, col=1)
fig.update_layout(template='plotly_dark', title='Częstotliwość zakupów ze względu na datę', width=950)
fig.show()

In [15]:
data_uk.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Sales
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34


In [16]:
# Łączna sprzedaż ze względu na datę
tmp = data_uk.groupby(data_uk['InvoiceDate'].dt.date)['Sales'].sum().reset_index()
tmp.columns = ['InvoiceDate', 'Sales']
tmp.head()

,InvoiceDate,Sales
0,2010-12-01,42030.85
1,2010-12-02,45622.08
2,2010-12-03,17512.44
3,2010-12-05,25458.85
4,2010-12-06,29007.74


In [17]:
fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.02)

trace1 = px.line(tmp, x='InvoiceDate', y='Sales', template='plotly_dark', color_discrete_sequence=['#03fcb5'])['data'][0]
trace2 = px.scatter(tmp, x='InvoiceDate', y='Sales', template='plotly_dark', color_discrete_sequence=['#03fcb5'])['data'][0]

fig.add_trace(trace1, row=1, col=1)
fig.add_trace(trace2, row=2, col=1)
fig.update_layout(template='plotly_dark', title='Łączna sprzedaż ze względu na datę', width=950)
fig.show()

### <a name='3'></a> Wyznacznie retencji klienta

In [18]:
# wydobycie unikalnych wartości CustomerID
data_user = pd.DataFrame(data['CustomerID'].unique(), columns=['CustomerID'])
data_user.head(3)

,CustomerID
0,17850.0
1,13047.0
2,12583.0


In [19]:
# wydobycie daty ostatniego zakupu dla każdego klienta
last_purchase = data_uk.groupby('CustomerID')['InvoiceDate'].max().reset_index()
last_purchase.columns = ['CustomerID', 'LastPurchaseDate']
last_purchase.head()

,CustomerID,LastPurchaseDate
0,12346.0,2011-01-18 10:17:00
1,12747.0,2011-12-07 14:34:00
2,12748.0,2011-12-09 12:20:00
3,12749.0,2011-12-06 09:56:00
4,12820.0,2011-12-06 15:12:00


In [20]:
last_purchase['LastPurchaseDate'].max()

Timestamp('2011-12-09 12:49:00')

In [21]:
last_purchase['LastPurchaseDate'].min()

Timestamp('2010-12-01 09:53:00')

In [22]:
# wyznaczenie retencji jako liczby dni od daty ostatniego kupna klienta do maksymalnej (ostatniej) daty kupna w danych
last_purchase['Retention'] = (last_purchase['LastPurchaseDate'].max() - last_purchase['LastPurchaseDate']).dt.days
last_purchase.head()

,CustomerID,LastPurchaseDate,Retention
0,12346.0,2011-01-18 10:17:00,325
1,12747.0,2011-12-07 14:34:00,1
2,12748.0,2011-12-09 12:20:00,0
3,12749.0,2011-12-06 09:56:00,3
4,12820.0,2011-12-06 15:12:00,2


In [23]:
last_purchase['Retention'].value_counts()

3      114
8       97
0       97
2       92
1       77
      ... 
243      1
370      1
285      1
295      1
174      1
Name: Retention, Length: 348, dtype: int64

In [24]:
px.histogram(last_purchase, x='Retention', template='plotly_dark', 
             width=950, height=500, title='Retention', nbins=100, 
             color_discrete_sequence=['#03fcb5'])

In [25]:
# połaczenie CustomerID oraz retencji
data_user = pd.merge(data_user, last_purchase, on='CustomerID')
data_user = data_user[['CustomerID', 'Retention']]
data_user.head()

,CustomerID,Retention
0,17850.0,301
1,13047.0,31
2,13748.0,95
3,15100.0,329
4,15291.0,25


In [26]:
px.scatter(data_user, x='CustomerID', y='Retention', template='plotly_dark', width=950,
           color_discrete_sequence=['#03fcb5'])

In [27]:
data_retention = data_user[['Retention']]
data_retention.head()

,Retention
0,301
1,31
2,95
3,329
4,25


In [28]:
# standaryzacja danych
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
data_user['RetentionScaled'] = scaler.fit_transform(data_retention)
data_user.head()

,CustomerID,Retention,RetentionScaled
0,17850.0,301,2.097649
1,13047.0,31,-0.596486
2,13748.0,95,0.042124
3,15100.0,329,2.377041
4,15291.0,25,-0.656356


In [29]:
px.scatter(data_user, x='CustomerID', y='RetentionScaled', template='plotly_dark', width=950,
           color_discrete_sequence=['#03fcb5'])

In [30]:
data_retention_scaled = data_user[['RetentionScaled']]
data_retention_scaled.head()

,RetentionScaled
0,2.097649
1,-0.596486
2,0.042124
3,2.377041
4,-0.656356


### <a name='4'></a> Retencja - KMeans

In [31]:
from sklearn.cluster import KMeans

wcss = []
for i in range(1, 10):
    kmeans = KMeans(n_clusters=i, max_iter=1000)
    kmeans.fit(data_retention_scaled)
    wcss.append(kmeans.inertia_)

wcss = pd.DataFrame(data=np.c_[range(1, 10), wcss], columns=['NumberOfClusters', 'WCSS'])
wcss

,NumberOfClusters,WCSS
0,1.0,3950.000000
1,2.0,814.413936
2,3.0,387.846731
3,4.0,221.025615
4,5.0,136.404793
5,6.0,94.777563
6,7.0,71.026115
7,8.0,54.248044
8,9.0,41.123462


In [32]:
px.line(wcss, x='NumberOfClusters', y='WCSS', template='plotly_dark', title='WCSS', 
        width=950, color_discrete_sequence=['#03fcb5'])

In [33]:
kmeans = KMeans(n_clusters=3, max_iter=1000)
kmeans.fit(data_retention_scaled)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=1000,
       n_clusters=3, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

In [34]:
data_user['Cluster'] = kmeans.labels_
data_user.head()

,CustomerID,Retention,RetentionScaled,Cluster
0,17850.0,301,2.097649,0
1,13047.0,31,-0.596486,1
2,13748.0,95,0.042124,2
3,15100.0,329,2.377041,0
4,15291.0,25,-0.656356,1


In [35]:
tmp = data_user.groupby('Cluster')['Retention'].describe()
tmp

,count,mean,std,min,25%,50%,75%,max
Cluster,,,,,,,,
0,558.0,294.433692,45.277184,225.0,256.0,287.0,329.00,373.0
1,2660.0,30.516541,25.207721,0.0,9.0,24.0,49.25,92.0
2,732.0,154.517760,38.145893,93.0,120.0,154.0,186.00,224.0


In [36]:
tmp = tmp['mean'].reset_index()
tmp.columns = ['Cluster', 'MeanRetention']
px.bar(tmp, x='Cluster', y='MeanRetention', template='plotly_dark', width=950, 
       height=400, color_discrete_sequence=['#03fcb5'])

In [37]:
px.scatter(data_user, x='CustomerID', y='Retention', color='Cluster', template='plotly_dark', 
           width=950, title='Wizualizacja klastrów')

In [38]:
tmp = data_user['Cluster'].value_counts().reset_index()
tmp.columns = ['Cluster', 'Count']
px.bar(tmp, x='Cluster', y='Count', template='plotly_dark', color_discrete_sequence=['#03fcb5'], width=950,
       title='Rozkład częstości klastrów')

### <a name='5'></a> Retencja - DBSCAN

In [39]:
from sklearn.cluster import DBSCAN

dbscan = DBSCAN(eps=0.03, min_samples=5)
dbscan.fit(data_retention_scaled)
clusters = dbscan.labels_
data_user['Cluster'] = clusters
data_user.head()

,CustomerID,Retention,RetentionScaled,Cluster
0,17850.0,301,2.097649,0
1,13047.0,31,-0.596486,1
2,13748.0,95,0.042124,1
3,15100.0,329,2.377041,0
4,15291.0,25,-0.656356,1


In [40]:
px.scatter(data_user, x='CustomerID', y='Retention', color='Cluster', template='plotly_dark', width=950,
           title='Wizualizacja klastrów')

### <a name='6'></a> Sprzedaż

In [41]:
data_sales = data_uk.groupby('CustomerID')['Sales'].sum().reset_index()
data_sales.head()

,CustomerID,Sales
0,12346.0,0.00
1,12747.0,4196.01
2,12748.0,29072.10
3,12749.0,3868.20
4,12820.0,942.34


In [42]:
data_user = pd.merge(data_user, data_sales, on='CustomerID')
data_user.head()

,CustomerID,Retention,RetentionScaled,Cluster,Sales
0,17850.0,301,2.097649,0,5288.63
1,13047.0,31,-0.596486,1,3079.10
2,13748.0,95,0.042124,1,948.25
3,15100.0,329,2.377041,0,635.10
4,15291.0,25,-0.656356,1,4596.51


In [43]:
scaler = StandardScaler()
data_user['SalesScaled'] = scaler.fit_transform(data_user[['Sales']])
data_user.head()

,CustomerID,Retention,RetentionScaled,Cluster,Sales,SalesScaled
0,17850.0,301,2.097649,0,5288.63,0.546024
1,13047.0,31,-0.596486,1,3079.10,0.208577
2,13748.0,95,0.042124,1,948.25,-0.116854
3,15100.0,329,2.377041,0,635.10,-0.164680
4,15291.0,25,-0.656356,1,4596.51,0.440321


In [44]:
px.scatter(data_user, x='CustomerID', y='Sales', template='plotly_dark',
           color_discrete_sequence=['#03fcb5'], title='Sprzedaż w rozbiciu na klienta')

In [45]:
px.scatter(data_user, x='CustomerID', y='SalesScaled', template='plotly_dark',
           color_discrete_sequence=['#03fcb5'], title='Sprzedaż w rozbiciu na klienta - dane przeskalowane')

In [46]:
data_sales_scaled = data_user[['SalesScaled']]
data_sales_scaled.head()

,SalesScaled
0,0.546024
1,0.208577
2,-0.116854
3,-0.164680
4,0.440321


### <a name='7'></a> Sprzedaż - KMeans

In [47]:
wcss = []
for i in range(1, 10):
    kmeans = KMeans(n_clusters=i, max_iter=1000)
    kmeans.fit(data_sales_scaled)
    wcss.append(kmeans.inertia_)

wcss = pd.DataFrame(data=np.c_[range(1, 10), wcss], columns=['NumberOfClusters', 'WCSS'])
wcss  

,NumberOfClusters,WCSS
0,1.0,3950.000000
1,2.0,1685.972621
2,3.0,595.548482
3,4.0,354.491493
4,5.0,221.261545
5,6.0,159.909637
6,7.0,104.454120
7,8.0,75.907281
8,9.0,53.525500


In [48]:
px.line(wcss, x='NumberOfClusters', y='WCSS', template='plotly_dark', color_discrete_sequence=['#03fcb5'], 
        width=950, title='WCSS')

In [49]:
kmeans = KMeans(n_clusters=3, max_iter=1000)
kmeans.fit(data_sales_scaled)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=1000,
       n_clusters=3, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

In [50]:
data_user['Cluster'] = kmeans.labels_
data_user['Cluster'] = data_user['Cluster'].astype(str)
data_user.head()

,CustomerID,Retention,RetentionScaled,Cluster,Sales,SalesScaled
0,17850.0,301,2.097649,0,5288.63,0.546024
1,13047.0,31,-0.596486,0,3079.10,0.208577
2,13748.0,95,0.042124,0,948.25,-0.116854
3,15100.0,329,2.377041,0,635.10,-0.164680
4,15291.0,25,-0.656356,0,4596.51,0.440321


In [51]:
kmeans.cluster_centers_

array([[-0.06065062],
       [33.63689221],
       [ 6.31619638]])

In [52]:
px.scatter(data_user, x='CustomerID', y='SalesScaled', color='Cluster', template='plotly_dark', width=950,
           title='Wizualizacja klastrów - dane przeskalowane')

### <a name='8'></a> Sprzedaż - DBSCAN

In [57]:
dbscan = DBSCAN(eps=0.5, min_samples=7)
dbscan.fit(data_sales_scaled)
clusters = dbscan.labels_
data_user['Cluster'] = clusters
data_user['Cluster'] = data_user['Cluster'].astype(str)
data_user.head()

,CustomerID,Retention,RetentionScaled,Cluster,Sales,SalesScaled
0,17850.0,301,2.097649,0,5288.63,0.546024
1,13047.0,31,-0.596486,0,3079.10,0.208577
2,13748.0,95,0.042124,0,948.25,-0.116854
3,15100.0,329,2.377041,0,635.10,-0.164680
4,15291.0,25,-0.656356,0,4596.51,0.440321


In [58]:
px.scatter(data_user, x='CustomerID', y='Sales', color='Cluster', template='plotly_dark', width=950,
           title='DBSCAN - Wizualizacja klastrów')

### <a name='9'></a> Retencja, sprzedaż - KMeans

In [59]:
data_user.head()

,CustomerID,Retention,RetentionScaled,Cluster,Sales,SalesScaled
0,17850.0,301,2.097649,0,5288.63,0.546024
1,13047.0,31,-0.596486,0,3079.10,0.208577
2,13748.0,95,0.042124,0,948.25,-0.116854
3,15100.0,329,2.377041,0,635.10,-0.164680
4,15291.0,25,-0.656356,0,4596.51,0.440321


In [60]:
px.scatter(data_user, x='RetentionScaled', y='SalesScaled', template='plotly_dark', width=950, 
           title='Retencja vs. Sprzedaż')

In [61]:
data_scaled = data_user[['RetentionScaled', 'SalesScaled']]
data_scaled.head()

,RetentionScaled,SalesScaled
0,2.097649,0.546024
1,-0.596486,0.208577
2,0.042124,-0.116854
3,2.377041,-0.164680
4,-0.656356,0.440321


In [62]:
wcss = []
for i in range(1, 10):
    kmeans = KMeans(n_clusters=i, max_iter=1000)
    kmeans.fit(data_scaled)
    wcss.append(kmeans.inertia_)

wcss = pd.DataFrame(data=np.c_[range(1, 10), wcss], columns=['NumberOfClusters', 'WCSS'])
wcss   

,NumberOfClusters,WCSS
0,1.0,7900.000000
1,2.0,4714.239225
2,3.0,2458.449395
3,4.0,1383.090602
4,5.0,953.199087
5,6.0,730.482572
6,7.0,566.351789
7,8.0,444.798855
8,9.0,360.958813


In [63]:
px.line(wcss, x='NumberOfClusters', y='WCSS', template='plotly_dark', color_discrete_sequence=['#03fcb5'], width=950,
        title='WCSS')

In [64]:
kmeans = KMeans(n_clusters=5, max_iter=1000)
kmeans.fit(data_scaled)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=1000,
       n_clusters=5, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

In [65]:
data_user['Cluster'] = kmeans.labels_
data_user['Cluster'] = data_user['Cluster'].astype(str)
data_user.head()

,CustomerID,Retention,RetentionScaled,Cluster,Sales,SalesScaled
0,17850.0,301,2.097649,2,5288.63,0.546024
1,13047.0,31,-0.596486,4,3079.10,0.208577
2,13748.0,95,0.042124,0,948.25,-0.116854
3,15100.0,329,2.377041,2,635.10,-0.164680
4,15291.0,25,-0.656356,4,4596.51,0.440321


In [66]:
px.scatter(data_user, x='RetentionScaled', y='SalesScaled', color='Cluster', template='plotly_dark', width=950,
           title='KMeans - Wizualizacja klastrów')

In [67]:
centroids = kmeans.cluster_centers_
centroids

array([[ 6.16600854e-01, -1.64763193e-01],
       [-8.70889226e-01,  3.36368922e+01],
       [ 2.02465229e+00, -2.02955850e-01],
       [-8.48160918e-01,  6.31619638e+00],
       [-6.02210172e-01, -4.16710543e-04]])

In [69]:
fig = px.scatter(data_user, x='RetentionScaled', y='SalesScaled', color='Cluster', template='plotly_dark', width=900,
                 title='KMeans - Wizualizacja klastrów + centroidy')
fig.add_trace(go.Scatter(x=centroids[:, 0], y=centroids[:, 1], mode='markers', marker_symbol='star',
                         marker_size=10, marker_color='white', showlegend=False))

In [70]:
desc = data_user.groupby('Cluster')[['Retention', 'Sales']].describe()
desc

Retention                         ...      Sales                        
            count        mean        std  ...        50%          75%        max
Cluster                                   ...                                   
0           738.0  152.903794  38.327273  ...     392.52     777.4825    7092.06
1             2.0    3.500000   4.949747  ...  221960.33  239199.4100  256438.49
2           564.0  293.684397  45.612310  ...     249.93     391.9725   21535.90
3            27.0    5.777778   9.082245  ...   36351.42   53489.7900   88125.38
4          2619.0   30.519664  24.930015  ...     911.26    2051.8900   21086.30

[5 rows x 16 columns]

In [71]:
tmp = pd.merge(desc['Retention'][['count', 'mean']].reset_index(), desc['Sales'][['mean']].reset_index(), on='Cluster',
         suffixes=('_Retention', '_Sales'))
tmp

,Cluster,count,mean_Retention,mean_Sales
0,0,738.0,152.903794,630.796695
1,1,2.0,3.500000,221960.330000
2,2,564.0,293.684397,384.475567
3,3,27.0,5.777778,43070.445185
4,4,2619.0,30.519664,1710.071987


In [72]:
px.bar(tmp, x='count', y='Cluster', hover_data=['mean_Retention', 'mean_Sales'], template='plotly_dark', 
       width=950, orientation='h', title='Rozkład klastrów')